Example link
# s3://nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/TotalPrecip_percentiles_201405.nc

In [1]:
import os
import json

# Load the config file and set it as JSON string in the environment
with open('../lithops_config.json', 'r') as f:
    config_data = json.load(f)

os.environ['LITHOPS_CONFIG'] = json.dumps(config_data)

from lithops import FunctionExecutor

def check_python(_):
    import sys
    return sys.version

with FunctionExecutor() as fexec:
    future = fexec.call_async(check_python, None)
    result = fexec.get_result([future])
    print(result)

2025-08-14 19:11:33,414 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-14 19:11:33,418 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-14 19:11:33,418 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-14 19:11:33,606 [INFO] invokers.py:119 -- ExecutorID 7141b5-0 | JobID A000 - Selected Runtime: /home/jovyan/casi-icechunk/.venv/bin/python 
2025-08-14 19:11:33,608 [INFO] invokers.py:186 -- ExecutorID 7141b5-0 | JobID A000 - Starting function invocation: check_python() - Total: 1 activations
2025-08-14 19:11:33,612 [INFO] invokers.py:225 -- ExecutorID 7141b5-0 | JobID A000 - View execution logs at /tmp/lithops-root/logs/7141b5-0-A000.log
2025-08-14 19:11:33,612 [INFO] executors.py:494 -- ExecutorID 7141b5-0 - Getting results from 1 function activations
2025-08-14 19:11:33,613 [INFO] wait.py:101 -- ExecutorID 7141b5-0 - Waiting for 1 function activations to complete
/home/jovyan/casi-icechunk/.venv/lib/python3.12/site-packages/tqdm/auto.p

3.12.10 (main, Apr  9 2025, 04:03:51) [Clang 20.1.0 ]


## First pass with fsspec.

In [74]:
import fsspec

fs = fsspec.filesystem('s3', anon=True)
files = fs.glob("s3://nasa-waterinsight/test/CASI/**/*1950*.nc")

In [75]:
files

['nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195001.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195002.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195003.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195004.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195005.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195006.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195007.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195008.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195009.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195010.nc',
 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195011.nc',
 'nasa-waterinsight/test/CASI/LS

In [76]:
# check netcdf format
import netCDF4 as nc
# Method 1: Using s3fs to create a file-like object
s3_path = 'nasa-waterinsight/test/CASI/LSM/HISTORICAL2/PERCENTILES/Snowf_percentiles_195001.nc'

with fs.open(s3_path, 'rb') as f:
    with nc.Dataset('dummy', mode='r', memory=f.read()) as ds:
        print(f"File format: {ds.file_format}")
        print(f"Data model: {ds.data_model}")

File format: NETCDF3_64BIT_OFFSET
Data model: NETCDF3_64BIT_OFFSET


In [98]:
from virtualizarr import open_virtual_mfdataset, open_virtual_dataset
from virtualizarr.registry import ObjectStoreRegistry
from virtualizarr.parsers import NetCDF3Parser
import obstore

## Produce a virtual dataset from the list of files
bucket = "s3://nasa-waterinsight"
store = obstore.store.from_url(bucket, region="us-west-2", skip_signature=True)

registry = ObjectStoreRegistry({bucket: store})

parser = NetCDF3Parser()

urls = ["s3://" + file for file in files]

def preprocess(ds:xr.Dataset)-> xr.Dataset:
    # parse the int date into datetime and add as coordinate
    time = xr.DataArray(pd.to_datetime(ds.date.values.astype(str), format='%Y%m'), dims=['time'])
    # drop the old coord
    ds = ds.drop('date')
    #expand the data vars and then assign the time coordinate
    ds = ds.expand_dims('time')
    ds = ds.assign_coords({'time':time})
    return ds

# vds_single = open_virtual_dataset(
#     urls[0],
#     parser=parser,
#     registry=registry,
# )
# vds_single

vds = open_virtual_mfdataset(
    urls,
    parser=parser,
    registry=registry,
    parallel="lithops",
    preprocess=preprocess
)

print(f"Virtual Dataset: {vds}")

2025-08-14 20:09:02,006 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-14 20:09:02,006 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-14 20:09:02,007 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-14 20:09:02,227 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-14 20:09:02,227 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-14 20:09:02,228 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-14 20:09:02,379 [INFO] invokers.py:119 -- ExecutorID 7141b5-18 | JobID M000 - Selected Runtime: /home/jovyan/casi-icechunk/.venv/bin/python 
2025-08-14 20:09:02,389 [INFO] invokers.py:186 -- ExecutorID 7141b5-18 | JobID M000 - Starting function invocation: _open_and_preprocess() - Total: 24 activations
2025-08-14 20:09:02,400 [INFO] invokers.py:225 -- ExecutorID 7141b5-18 | JobID M000 - View execution logs at /tmp/lithops-root/logs/7141b5-18-M000.log
2025-08-14 20:09:02,405 [INFO] executors.py:494

NotImplementedError: ManifestArrays can't be converted into numpy arrays or pandas Index objects

In [92]:
vds_list = [open_virtual_dataset(
    url,
    parser=parser,
    registry=registry,
) for url in urls]

In [97]:
vds_list_single_var = [vds for vds in vds_list if 'Snowf_percentiles' in vds.data_vars]
# xr.combine_by_coords(vds_list_single_var)
xr.concat(vds_list_single_var, dim='ncl0')

/tmp/ipykernel_2474/373596303.py:3: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  xr.concat(vds_list_single_var, dim='ncl0')


<xarray.Dataset> Size: 5MB
Dimensions:            (ncl0: 12, percentile: 5, lat: 152, lon: 132)
Coordinates:
  * percentile         (percentile) int32 20B 10 25 50 75 90
  * lat                (lat) float32 608B 23.95 24.05 24.15 ... 38.95 39.05
  * lon                (lon) float32 528B -109.5 -109.5 -109.3 ... -96.55 -96.45
Dimensions without coordinates: ncl0
Data variables:
    date               (ncl0) int32 48B ManifestArray<shape=(12,), dtype=int3...
    Snowf_percentiles  (ncl0, percentile, lat, lon) float32 5MB ManifestArray...

In [93]:
vds_list

[<xarray.Dataset> Size: 402kB
 Dimensions:            (percentile: 5, lat: 152, lon: 132, ncl0: 1)
 Coordinates:
   * percentile         (percentile) int32 20B 10 25 50 75 90
   * lat                (lat) float32 608B 23.95 24.05 24.15 ... 38.95 39.05
   * lon                (lon) float32 528B -109.5 -109.5 -109.3 ... -96.55 -96.45
 Dimensions without coordinates: ncl0
 Data variables:
     date               (ncl0) int32 4B ManifestArray<shape=(1,), dtype=int32,...
     Snowf_percentiles  (percentile, lat, lon) float32 401kB ManifestArray<sha...,
 <xarray.Dataset> Size: 402kB
 Dimensions:            (percentile: 5, lat: 152, lon: 132, ncl0: 1)
 Coordinates:
   * percentile         (percentile) int32 20B 10 25 50 75 90
   * lat                (lat) float32 608B 23.95 24.05 24.15 ... 38.95 39.05
   * lon                (lon) float32 528B -109.5 -109.5 -109.3 ... -96.55 -96.45
 Dimensions without coordinates: ncl0
 Data variables:
     date               (ncl0) int32 4B ManifestArray<sha

In [91]:
with fsspec.open(urls[0]) as f:
    ds = xr.open_dataset(f)
ds_new = preprocess(ds)
ds_new

/tmp/ipykernel_2474/2325006497.py:20: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds = ds.drop('date')


<xarray.Dataset> Size: 402kB
Dimensions:            (time: 1, percentile: 5, lat: 152, lon: 132)
Coordinates:
  * percentile         (percentile) int32 20B 10 25 50 75 90
  * lat                (lat) float32 608B 23.95 24.05 24.15 ... 38.95 39.05
  * lon                (lon) float32 528B -109.5 -109.5 -109.3 ... -96.55 -96.45
  * time               (time) datetime64[ns] 8B 1950-01-01
Data variables:
    Snowf_percentiles  (time, percentile, lat, lon) float32 401kB nan ... 27.47

In [87]:
ds.expand_dims('time')

<xarray.Dataset> Size: 402kB
Dimensions:            (time: 1, ncl0: 1, percentile: 5, lat: 152, lon: 132)
Coordinates:
  * percentile         (percentile) int32 20B 10 25 50 75 90
  * lat                (lat) float32 608B 23.95 24.05 24.15 ... 38.95 39.05
  * lon                (lon) float32 528B -109.5 -109.5 -109.3 ... -96.55 -96.45
Dimensions without coordinates: time, ncl0
Data variables:
    date               (time, ncl0) int32 4B 195001
    Snowf_percentiles  (time, percentile, lat, lon) float32 401kB nan ... 27.47

## I want to do this with obstore tho

In [ ]:
import obstore

In [ ]:
store = obstore.store.S3Store(
    bucket='s3://nasa-watersight',
    skip_signature=True,

In [4]:
store.head('test/CASI/LSM/HISTORICAL2/PERCENTILES/TotalPrecip_percentiles_201405.nc')

GenericError: Generic S3 error: Error performing HEAD https://s3.us-west-2.amazonaws.com/s3://nasa-watersight/test/CASI/LSM/HISTORICAL2/PERCENTILES/TotalPrecip_percentiles_201405.nc in 7.921867ms - Server returned non-2xx status code: 400 Bad Request: 

Debug source:
Generic {
    store: "S3",
    source: RetryError(
        RetryErrorImpl {
            method: HEAD,
            uri: Some(
                https://s3.us-west-2.amazonaws.com/s3://nasa-watersight/test/CASI/LSM/HISTORICAL2/PERCENTILES/TotalPrecip_percentiles_201405.nc,
            ),
            retries: 0,
            max_retries: 10,
            elapsed: 7.921867ms,
            retry_timeout: 180s,
            inner: Status {
                status: 400,
                body: Some(
                    "",
                ),
            },
        },
    ),
}